# Environment

## Imports

In [2]:
:dep polars = { version = "0.46", features = [ "full" ] }

In [4]:
use polars::prelude::*;
use polars::df;

# Problem 1

## a)

Initialize data.

In [6]:
let df: DataFrame = df!(
    "rate_type" => ["cash", "cash", "forwards", "forwards", "forwards", "swaps", "swaps", "swaps", "swaps", "swaps"],
    "maturity" => [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0],
    "inputs" => [0.03180, 0.03222, 0.03261, 0.03290, 0.03345, 0.03405, 0.03442, 0.03350, 0.03300, 0.03541],
    "bond_cash_flow" => [3, 3, 3, 3, 3, 3, 3, 3, 103, 0],
).unwrap();
df

shape: (10, 4)
┌───────────┬──────────┬─────────┬────────────────┐
│ rate_type ┆ maturity ┆ inputs  ┆ bond_cash_flow │
│ ---       ┆ ---      ┆ ---     ┆ ---            │
│ str       ┆ f64      ┆ f64     ┆ i32            │
╞═══════════╪══════════╪═════════╪════════════════╡
│ cash      ┆ 1.0      ┆ 0.0318  ┆ 3              │
│ cash      ┆ 2.0      ┆ 0.03222 ┆ 3              │
│ forwards  ┆ 3.0      ┆ 0.03261 ┆ 3              │
│ forwards  ┆ 4.0      ┆ 0.0329  ┆ 3              │
│ forwards  ┆ 5.0      ┆ 0.03345 ┆ 3              │
│ swaps     ┆ 6.0      ┆ 0.03405 ┆ 3              │
│ swaps     ┆ 7.0      ┆ 0.03442 ┆ 3              │
│ swaps     ┆ 8.0      ┆ 0.0335  ┆ 3              │
│ swaps     ┆ 9.0      ┆ 0.033   ┆ 103            │
│ swaps     ┆ 10.0     ┆ 0.03541 ┆ 0              │
└───────────┴──────────┴─────────┴────────────────┘

Place the inputs into their appropriate curves.

In [7]:
let lf = df
    .lazy()
    .with_column(
        when(col("rate_type").eq(lit("cash")))
            .then(col("inputs"))
            .otherwise(lit(NULL))
            .alias("zero_curve"),
    )
    .with_column(
        when(col("rate_type").eq(lit("forwards")))
            .then(col("inputs"))
            .otherwise(lit(NULL))
            .alias("forward_curve"),
    )
    .with_column(
        when(col("rate_type").eq(lit("swaps")))
            .then(col("inputs"))
            .otherwise(lit(NULL))
            .alias("par_curve"),
    );
lf.collect().unwrap()

shape: (10, 7)
┌───────────┬──────────┬─────────┬────────────────┬────────────┬───────────────┬───────────┐
│ rate_type ┆ maturity ┆ inputs  ┆ bond_cash_flow ┆ zero_curve ┆ forward_curve ┆ par_curve │
│ ---       ┆ ---      ┆ ---     ┆ ---            ┆ ---        ┆ ---           ┆ ---       │
│ str       ┆ f64      ┆ f64     ┆ i32            ┆ f64        ┆ f64           ┆ f64       │
╞═══════════╪══════════╪═════════╪════════════════╪════════════╪═══════════════╪═══════════╡
│ cash      ┆ 1.0      ┆ 0.0318  ┆ 3              ┆ 0.0318     ┆ null          ┆ null      │
│ cash      ┆ 2.0      ┆ 0.03222 ┆ 3              ┆ 0.03222    ┆ null          ┆ null      │
│ forwards  ┆ 3.0      ┆ 0.03261 ┆ 3              ┆ null       ┆ 0.03261       ┆ null      │
│ forwards  ┆ 4.0      ┆ 0.0329  ┆ 3              ┆ null       ┆ 0.0329        ┆ null      │
│ forwards  ┆ 5.0      ┆ 0.03345 ┆ 3              ┆ null       ┆ 0.03345       ┆ null      │
│ swaps     ┆ 6.0      ┆ 0.03405 ┆ 3              ┆ nul

In [9]:
let lf = lf.with_column(when(col("zero_curve").is_not_null()).then(col("zero_curve").add(lit(1.0)).pow(col("maturity"))).otherwise(lit(NULL)).alias("forward_curve"));
lf.collect().unwrap();

Error: cannot find value `lf` in this scope

Error: the method `add` exists for enum `Expr`, but its trait bounds were not satisfied